# Memory (대화 내용 기억)
```04_memory.ipynb```
- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다(stateless)
- 이전 대화내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
- 단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
- 장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어 넣는것.
2. 이때 대화가 길어지면 토큰수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
- 요약
- 적정 길이에서 앞부분 자르기
- 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)

# ```ConversationBufferMemory```
- 메시지 저장 -> 변수에서 추출 가능

In [7]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

{'chat_history': []}

In [8]:
from operator import itemgetter
# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

In [9]:
chain = runnable | prompt | llm

In [10]:
res = chain.invoke({'input': '만나서 반가워'})

In [11]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [12]:
my_message = '내 이름 뭐라고?'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [13]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반가워', additional_kwargs={}, response_metadata={}),
  AIMessage(content='저도 만나서 반가워요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름 뭐라고?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만, 제가 당신의 이름을 알지 못해요. 이름을 알려주시면 기억해둘게요!', additional_kwargs={}, response_metadata={})]}

In [17]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# Prompt
prompt = ChatPromptTemplate(
    [
        ('system', '넌 좀 틱틱대는 챗봇이야. '),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

chain = runnable | prompt | llm | StrOutputParser()

# 메세지 스트리밍 하는 함수
def stream_msg(chain, msg):
    full_msg = ''
    print('AI: ', end='')
    for token in chain.stream({'input': msg}):
        full_msg += token
        print(token, end='', flush=True)
    print('\n---')
    return full_msg

input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    print('인간: ', input_msg)
    output_msg = stream_msg(chain, input_msg)
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg}
    )
    time.sleep(0.5)

인간:  난 학생! 넌 누구니?
AI: 나는 너를 도와주는 챗봇이야. 궁금한 거 있으면 언제든 말해!
---
인간:  어떠한 원리로 넌 작동하는 거니?
AI: 내 작동 원리는 인공지능과 자연어 처리 기술을 기반으로 해. 많은 텍스트 데이터를 학습해서 너의 질문이나 요청에 적절한 답변을 만들어내는 거지. 좀 더 쉽게 말하면, 너가 말하는 내용을 이해하고 그에 맞는 답을 찾기 위해 수많은 정보를 분석하는 거라고 보면 돼.
---
인간:  그럼 만약 내가 너에게 대한민국의 수도는 어디야? 라고 물으면 어떻게 답할래?
AI: 대한민국의 수도는 서울이야.
---
인간:  서울 이라는 답은 어디서 찾아 온거야?
AI: 그 답은 내가 학습한 방대한 텍스트 데이터에서 얻은 정보야. 대한민국의 수도에 관한 일반적인 사실이 여러 자료에 반복해서 등장하기 때문에, 그 정보를 기억하고 있어서 너에게 알려줄 수 있는 거지. 즉, 내가 학습한 데이터 내에 있는 공통적이고 신뢰할 만한 정보를 바탕으로 답변하는 거야.
---
인간:  너는 인공지능이라고 했는데 모델명 같은 것은 없어?
AI: 내 모델명은 GPT-4야. 너가 궁금해할 만한 기술적 정보도 알고 있으니, 더 궁금한 게 있으면 언제든 물어봐!
---
인간:  일단 지금은 정지
AI: 알겠어. 필요하면 언제든 다시 말해줘!
---
인간:  
AI: 말이 없네? 궁금한 거 있으면 언제든 말해줘!
---
인간:  정지
AI: 알았어. 언제든 다시 필요하면 말해줘!
---


In [18]:
# 레포트 작성 챗봇
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# Output parser 정의
class Report(BaseModel):
    title: str = Field(..., description='보고서의 제목')
    summary: str = Field(..., description='보고서 요약본')
    content_kr: str = Field(..., description='한국어로 작성된 보고서의 내용(1000자 이내)')
    content_jp: str = Field(..., description='일본어로 작성된 보고서의 내용(1000자 이내)')

parser = PydanticOutputParser(pydantic_object=Report)

# Prompt에 format instructions 추가
prompt = ChatPromptTemplate(
    [
        ('system', '넌 보고서 작성에 특화된 챗봇이야. '
                   '반드시 지정된 형식에 맞춰 JSON으로 답변해.\n\nFORMAT INSTRUCTION: {format_instructions}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

# 체인 구성
chain = runnable | prompt | llm | parser

input_msg = ''

# 사용자가 ('quit', '정지', '그만', '') 중에 하나를 입력하면 대화 종료
while 1:
    input_msg = input()
    if input_msg  in ('quit', '정지', '그만', ''):
        break

    print('인간: ', input_msg)
    output_msg = chain.invoke({'input': input_msg})
    pprint(output_msg.model_dump())
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg.model_dump_json()}
    )

인간:  지금 집을 마련을 해야해? 서울에 살고 싶고 20억원 이하의 새로운 집이 필요한데 어디가 좋을까?
{'content_jp': '現在、ソウルの不動産市場では、さまざまな新築住宅が供給されており、江南区、江北区、江西区、松坡区などの主要地域で20億ウォン以下の新築アパートを見つけることができます。江南区は交通と教育インフラが優れている反面、価格が高いため競争が激しいです。江北区と江西区は比較的価格が低く、新築物件も豊富でコストパフォーマンスの良い選択肢となります。また、竜山区や麻浦区も開発の好材料と交通改善により人気が高まっており、おすすめです。具体的な選択は、勤務先の場所や生活の便、将来の開発計画などを考慮して決めることが重要です。',
 'content_kr': '현재 서울 부동산 시장은 다양한 신축 주택이 공급되고 있으며, 강남권, 강북권, 강서구, 송파구 등 주요 '
               '지역에서 20억 이하의 신축 아파트를 찾을 수 있습니다. 강남권은 교통과 교육 인프라가 뛰어나지만 가격이 높아 '
               '경쟁이 치열합니다. 강북권과 강서구는 상대적으로 가격이 낮고 신축 물량이 풍부하여 가성비 좋은 선택이 될 수 '
               '있습니다. 또한, 용산구와 마포구도 개발 호재와 교통 개선으로 인기가 높아지고 있어 추천드립니다. 구체적인 '
               '선택은 본인의 직장 위치, 생활 편의시설, 향후 개발 계획 등을 고려하여 결정하는 것이 좋습니다.',
 'summary': '서울에서 20억 이하의 신규 주택을 찾는 고객을 위한 추천 및 시장 분석 보고서입니다.',
 'title': '서울 내 20억 이하 신규 주택 추천'}
